# Face Recognition Scanner

This notebook performs face recognition using template matching and PCA features with cosine_similarity checking.

In [ ]:
import cv2
import json
import os
import numpy as np
import pickle
import argparse
import glob
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

In [ ]:
class MultiModelFaceScanner:
    def __init__(self):
        """
        Initialize multi-model face scanner
        """
        self.models = {}
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
    def load_all_models(self):
        """
        Load all available models from faces/lock_version/*/face_model.pkl
        """
        model_pattern = "faces/lock_version/*/face_model.pkl"
        model_paths = glob.glob(model_pattern)
        
        if not model_paths:
            print(f"No models found matching pattern: {model_pattern}")
            return False
        
        print(f"Found {len(model_paths)} model(s):")
        
        for model_path in model_paths:
            # Extract person name from path
            person_name = os.path.basename(os.path.dirname(model_path))
            
            try:
                # Load model
                with open(model_path, 'rb') as f:
                    model_data = pickle.load(f)
                
                # Load detection data
                detection_json_path = os.path.join(os.path.dirname(model_path), f"{person_name}_faces_detection.json")
                detection_data = None
                if os.path.exists(detection_json_path):
                    with open(detection_json_path, 'r', encoding='utf-8') as f:
                        detection_data = json.load(f)
                
                # Load template image
                template_image = None
                if detection_data and detection_data['faces']:
                    first_face = detection_data['faces'][0]
                    template_path = first_face['image_path']
                    if os.path.exists(template_path):
                        template_image = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
                
                self.models[person_name] = {
                    'model_data': model_data,
                    'detection_data': detection_data,
                    'template_image': template_image,
                    'model_path': model_path
                }
                
                face_count = len(model_data['face_features']) if model_data else 0
                print(f"  - {person_name}: {face_count} faces")
                
            except Exception as e:
                print(f"  - Failed to load {person_name}: {e}")
        
        print(f"Successfully loaded {len(self.models)} model(s)")
        return len(self.models) > 0
    
    def extract_face_features(self, face_img, model_data):
        """
        Extract face features using specific model
        """
        if len(face_img.shape) == 3:
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = face_img
        
        resized = cv2.resize(gray, (64, 64))
        flattened = resized.flatten().reshape(1, -1)
        
        scaled = model_data['scaler'].transform(flattened)
        features = model_data['pca'].transform(scaled)
        
        return features[0]
    
    def recognize_face_with_model(self, face_features, model_data, threshold=0.7):
        """
        Recognize face using specific model
        """
        similarities = cosine_similarity([face_features], model_data['face_features'])[0]
        max_idx = np.argmax(similarities)
        max_similarity = similarities[max_idx]
        
        if max_similarity >= threshold:
            person_id = model_data['face_labels'][max_idx]
            person_name = "unknown"
            for name, pid in model_data['person_id_map'].items():
                if pid == person_id:
                    person_name = name
                    break
            return person_id, person_name, max_similarity
        else:
            return -1, "unknown", max_similarity
    
    def recognize_face_all_models(self, face_img, threshold=0.7):
        """
        Recognize face using all loaded models and return best match
        """
        best_result = None
        best_confidence = 0.0
        best_person = "unknown"
        
        for person_name, model_info in self.models.items():
            model_data = model_info['model_data']
            if model_data is None:
                continue
            
            try:
                features = self.extract_face_features(face_img, model_data)
                person_id, recognized_name, confidence = self.recognize_face_with_model(features, model_data, threshold)
                
                if confidence > best_confidence:
                    best_confidence = confidence
                    best_person = recognized_name if recognized_name != "unknown" else person_name
                    best_result = (person_id, best_person, confidence)
            
            except Exception as e:
                print(f"Error recognizing with model {person_name}: {e}")
                continue
        
        if best_result:
            return best_result
        else:
            return -1, "unknown", 0.0
    
    def process_live_camera(self, show_preview=True):
        """
        Process live camera for real-time face recognition using all models
        """
        if not self.models:
            print("Error: No models loaded!")
            return
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Could not open camera")
            return
        
        print("Starting live face recognition with multiple models...")
        print("Press 'q' to quit")
        
        frame_count = 0
        recognition_results = []
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to read frame from camera")
                break
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detect faces using Haar cascade
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
            
            for (x, y, w, h) in faces:
                # Extract face image
                face_img = frame[y:y+h, x:x+w]
                
                # Recognize face using all models
                person_id, person_name, confidence = self.recognize_face_all_models(face_img)
                
                # Draw result
                color = (0, 255, 0) if person_name != "unknown" else (0, 0, 255)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                
                # Add label
                label = f"{person_name} ({confidence:.2f})"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Record result
                result = {
                    'frame_number': frame_count,
                    'person_name': person_name,
                    'confidence': confidence,
                    'x': x, 'y': y, 'width': w, 'height': h
                }
                recognition_results.append(result)
            
            # Show preview
            if show_preview:
                cv2.imshow('Multi-Model Face Recognition', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            frame_count += 1
        
        # Clean up resources
        cap.release()
        cv2.destroyAllWindows()
        
        print("\nLive recognition stopped!")
        return recognition_results

## User Input Section

Please provide the required information:

In [6]:
# Get user input
person_name = input("Enter person name: ")

# Choose input mode
print("\nChoose input mode:")
print("1. Video file")
print("2. Live camera")
mode_choice = input("Enter your choice (1 or 2): ")

is_live_mode = mode_choice == '2'
video_input = None

if not is_live_mode:
    video_input = input("Enter video file path: ")

print(f"Person name: {person_name}")
print(f"Mode: {'Live camera' if is_live_mode else 'Video file'}")
if video_input:
    print(f"Video input: {video_input}")


Choose input mode:
1. Video file
2. Live camera
Person name: Joseph_Lai
Mode: Video file
Video input: Joseph_Lai.mp4


In [7]:
# Configure paths based on person name
base_dir = f"faces/lock_version/{person_name}"
model_path = f"{base_dir}/face_model.pkl"
detection_json_path = f"{base_dir}/{person_name}_faces_detection.json"
output_video = f"{base_dir}/recognition_output.mp4"
video_input = f"videos/{video_input}"

print(f"Model path: {model_path}")
print(f"Detection JSON: {detection_json_path}")
print(f"Output video: {output_video}")
print(f"Video input: {video_input}")


Model path: faces/lock_version/Joseph_Lai/face_model.pkl
Detection JSON: faces/lock_version/Joseph_Lai/Joseph_Lai_faces_detection.json
Output video: faces/lock_version/Joseph_Lai/recognition_output.mp4
Video input: videos/Joseph_Lai.mp4


In [8]:
# Check necessary files
if not os.path.exists(model_path):
    print(f"Error: Model file {model_path} not found!")
    print("Please run train-v2.py first to train the model.")
else:
    print("Model file found.")

if not os.path.exists(detection_json_path):
    print(f"Error: Detection JSON {detection_json_path} not found!")
    print("Please run detection-v2.py first to generate detection data.")
else:
    print("Detection JSON found.")

if not is_live_mode and video_input and not os.path.exists(video_input):
    print(f"Error: Video file {video_input} not found!")
elif not is_live_mode and video_input:
    print("Video file found.")

Model file found.
Detection JSON found.
Video file found.


In [ ]:
# Create scanner and load model
scanner = MultiModelFaceScanner()

# Load all available models
if not scanner.load_all_models():
    print("Failed to load any models!")
    print("Please run train-v2.py first to train models.")
    return

PCA model loaded: 77 faces, 1 persons
Detection data loaded: 77 detected faces
Template image loaded: faces/lock_version/Joseph_Lai\face_000000_frame_000000.jpg
Model and data loaded successfully!


In [ ]:
# Process video or live camera
if is_live_mode:
    print("\nStarting live face recognition...")
    print("Press 'q' to quit")
    results = scanner.process_live_camera(show_preview=True)
else:
    print("\nStarting video face recognition...")
    print("Press 'q' to quit preview window")
    # Note: Video processing for MultiModelFaceScanner not implemented yet
    print("Video processing with multi-model scanner not implemented yet.")
    print("Please use --live mode for now.")
    return
    
if results:
    print(f"\nRecognition summary:")
    # Count recognition results
    person_counts = {}
    for result in results:
        name = result['person_name']
        person_counts[name] = person_counts.get(name, 0) + 1
    
    for name, count in person_counts.items():
        print(f"  {name}: {count} detections")


Starting face recognition...
Press 'q' to quit preview window
Processing video: videos/Joseph_Lai.mp4
Resolution: 480x848, FPS: 30.0, Total frames: 131


Progress: 76.3% (100/131 frames)

Recognition completed!
Total recognitions: 109
Results saved to: faces/lock_version/Joseph_Lai/recognition_results.json
Output video saved to: faces/lock_version/Joseph_Lai/recognition_output.mp4

Recognition summary:
  Joseph_Lai: 87 detections
  unknown: 22 detections
